## 1. 환경 설정

`(1) Env 환경변수`

In [16]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 라이브러리`

In [17]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

## 2. 도구 호출 (Tool Calling)
- 도구 호출은 LLM이 특정 작업을 수행하기 위해 외부 기능을 호출하는 기능
- 이를 통해 LLM은 외부 API 통합 등 더 복잡한 작업을 수행할 수 있음 

### 2-1. 랭체인 내장 도구
- Tavily 웹 검색 도구 (예시)

`(1) 도구(tool) 정의하기`

In [18]:
from langchain_community.tools import TavilySearchResults
from langchain_tavily.tavily_search import TavilySearch
# 검색할 쿼리 설정
query = "스테이크와 어울리는 와인을 추천해주세요."

# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
#web_search = TavilySearchResults(max_results=2)
web_search = TavilySearch(max_results=2)

# 웹 검색 실행
search_results = web_search.invoke(query)
pprint(search_results)
# 검색 결과 출력
for result in search_results:
    print(result)  
    print("-" * 100)
print(search_results['results'])
    

{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': '스테이크와 어울리는 와인을 추천해주세요.',
 'response_time': 2.02,
 'results': [{'content': '스테이크와 어울리는 최고의 와인 추천 :: 시난 데일리 와인 블로그 내 검색  삭제 스테이크와 '
                         '어울리는 최고의 와인 추천 또한, 고기의 숙성 여부나 소스의 종류도 와인 선택에 중요한 요소가 '
                         '될 수 있다. 아르헨티나의 말벡(Malbec)은 육즙이 풍부한 스테이크와 궁합이 좋으며, '
                         '탄닌이 적절하고 과실미가 뛰어나 부담 없이 즐길 수 있다. 레드 와인은 보통 16~18°C '
                         '정도에서 서빙해야 풍미가 가장 잘 살아나며, 너무 차갑거나 따뜻하면 와인의 본래 맛이 달라질 '
                         '수 있다. 특히, 탄닌이 강한 보르도 와인이나 바롤로 같은 와인은 디캔팅을 통해 부드러운 '
                         '질감을 살릴 수 있다. 스테이크와 와인을 더욱 완벽하게 즐기려면 와인 잔 선택도 중요하다. '
                         '레드 와인은 볼이 넓은 글라스에서 마셔야 와인의 풍미가 극대화되며, 향을 더욱 깊이 느낄 수 '
                         '있다. 립아이와 같은 기름진 부위는 까베르네 소비뇽과 잘 어울리며, 부드러운 필레미뇽에는 피노 '
                         '누아가 좋은 선택이 될 수 있다.',
              'raw_content': None,
              'score': 0.9287263,
              'title': '스테이크와 어울리는 최고의 와인 추천 

In [19]:
# 도구 속성
print("자료형: ")
print(type(web_search))
print("-"*100)

print("name: ")
print(web_search.name)
print("-"*100)

print("description: ")
pprint(web_search.description)
print("-"*100)

print("schema: ")
pprint(web_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_tavily.tavily_search.TavilySearch'>
----------------------------------------------------------------------------------------------------
name: 
tavily_search
----------------------------------------------------------------------------------------------------
description: 
('A search engine optimized for comprehensive, accurate, and trusted results. '
 'Useful for when you need to answer questions about current events. It not '
 'only retrieves URLs and snippets, but offers advanced search depths, domain '
 'management, time range filters, and image search, this tool delivers '
 'real-time, accurate, and citation-backed results.Input should be a search '
 'query.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input for [TavilySearch]',
 'properties': {'exclude_domains': {'anyOf': [{'items': {'type': 'string'},
                                               'type': 'array'},
        

`(2) 도구(tool) 호출하기`

In [20]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# 웹 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=[web_search])

In [21]:
# 도구 호출이 필요 없는 LLM 호출을 수행
query = "안녕하세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 813, 'total_tokens': 835, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BiwmSjbjCXVASosxoMDpQ1rCN6trp', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--6fc1c379-ff47-4a91-b575-dec679a5c601-0', usage_metadata={'input_tokens': 813, 'output_tokens': 22, 'total_tokens': 835, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})
----------------------------------------------------------------------------------------------------
'안녕하세요! 무엇을 도와드릴까요?'
-----------------------------------------------------------------------

In [22]:
from pprint import pprint

# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = llm_with_tools.invoke(query)

# AIMessage의 속성 확인
pprint(vars(ai_msg))


{'additional_kwargs': {'refusal': None,
                       'tool_calls': [{'function': {'arguments': '{"query":"wine '
                                                                 'pairing with '
                                                                 'steak"}',
                                                    'name': 'tavily_search'},
                                       'id': 'call_ZD6L5LWCWGWb5d7JjhMz3rYC',
                                       'type': 'function'}]},
 'content': '',
 'example': False,
 'id': 'run--f51250a0-82ff-45fc-ac49-6a0bd3da9ff4-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'tool_calls',
                       'id': 'chatcmpl-BiwmTDWqkkRRvrMmURoHVdizAzizb',
                       'logprobs': None,
                       'model_name': 'gpt-3.5-turbo-0125',
                       'service_tier': 'default',
                       'system_fingerprint': None,
                       'token_usage': {'compl

In [23]:

pprint(ai_msg)
print("#" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("*" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZD6L5LWCWGWb5d7JjhMz3rYC', 'function': {'arguments': '{"query":"wine pairing with steak"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 829, 'total_tokens': 848, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BiwmTDWqkkRRvrMmURoHVdizAzizb', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f51250a0-82ff-45fc-ac49-6a0bd3da9ff4-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'wine pairing with steak'}, 'id': 'call_ZD6L5LWCWGWb5d7JjhMz3rYC', 'type': 'tool_call'}], usage_metadata={'input_tokens': 829, 'output_tokens': 19, 'total_tokens': 848, 

In [24]:
tool_call = ai_msg.tool_calls[0]

print(tool_call)

{'name': 'tavily_search', 'args': {'query': 'wine pairing with steak'}, 'id': 'call_ZD6L5LWCWGWb5d7JjhMz3rYC', 'type': 'tool_call'}


`(3) 도구(tool) 실행하기`

#### 방법 1: 직접 도구 호출 처리한 결과로 ToolMessage 객체 생성

* 이 방법은 AI 메시지에서 첫 번째 도구 호출을 가져와 직접 처리한다.
* 'args'를 사용하여 도구를 호출하고 결과를 얻는다.
* 도구 호출 결과를 사용하여 ToolMessage 객체를 생성한다.
* 도구 호출의 ID와 이름을 포함하여 더 구조화된 ToolMessage 메시지를 만든다.

In [25]:
tool_output = web_search.invoke(tool_call["args"])
print(f"{tool_call['name']} 호출 결과:")
print("-" * 100)

pprint(tool_output)

tavily_search 호출 결과:
----------------------------------------------------------------------------------------------------
{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': 'wine pairing with steak',
 'response_time': 1.9,
 'results': [{'content': 'The Handy Guide to Wine and Steak Pairing | Wine '
                         'Folly The Handy Guide to Wine and Steak Pairing With '
                         'so many different cuts and ways to prepare a steak, '
                         'you can discover a few go-to dry red wines or you '
                         'can dive deep into pairing nuance. The rule of thumb '
                         'when pairing with steak is to choose dry red wines – '
                         'leaner cuts of meat pair with lighter wines, while '
                         'richer, fattier cuts pair up with high tannin wines '
                         'that can cut through the fat. Why they work: The '
                         'Strip is flavorf

In [26]:

from langchain_core.messages import ToolMessage

tool_message = ToolMessage(
    content=tool_output,
    tool_call_id=tool_call["id"],
    name=tool_call["name"]
)

pprint(tool_message.content)

("{'query': 'wine pairing with steak', 'follow_up_questions': None, 'answer': "
 "None, 'images': [], 'results': [{'title': 'The Handy Guide to Wine and Steak "
 "Pairing', 'url': "
 "'https://winefolly.com/wine-pairing/the-handy-guide-to-wine-and-steak-pairing/', "
 "'content': 'The Handy Guide to Wine and Steak Pairing | Wine Folly The Handy "
 'Guide to Wine and Steak Pairing With so many different cuts and ways to '
 'prepare a steak, you can discover a few go-to dry red wines or you can dive '
 'deep into pairing nuance. The rule of thumb when pairing with steak is to '
 'choose dry red wines – leaner cuts of meat pair with lighter wines, while '
 'richer, fattier cuts pair up with high tannin wines that can cut through the '
 'fat. Why they work: The Strip is flavorful and a thicker grain cut of beef, '
 'making it useful for several different methods of cooking, but it needs a '
 "wine that can complement the flavor AND cut through the fat.', 'score': "
 "0.92763275, 'raw_conten

#### 방법 2: 도구 직접 호출하여 바로 ToolMessage 객체 생성

* 이 방법은 도구를 직접 호출하여 ToolMessage 객체를 생성한다.
* 가장 간단하고 직관적인 방법으로, LangChain의 추상화를 활용한다.

In [27]:
# tool_call  {'name': 'tavily_search_results_json', 'args': {'query': 'wine pairing with steak'}, 'id': 'call_LrHyxTadqHDjW7J6LOWEaoSi', 'type': 'tool_call'}
tool_message = web_search.invoke(tool_call)
print(type(tool_message))

# 특정 속성들만 확인
print("\n=== 주요 속성들 ===")
attributes = ['content', 'tool_call_id', 'name', 'type', 'additional_kwargs']
for attr in attributes:
    if hasattr(tool_message, attr):
        print(f"{attr}: {getattr(tool_message, attr)}")


<class 'langchain_core.messages.tool.ToolMessage'>

=== 주요 속성들 ===
content: {"query": "wine pairing with steak", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "The Handy Guide to Wine and Steak Pairing", "url": "https://winefolly.com/wine-pairing/the-handy-guide-to-wine-and-steak-pairing/", "content": "The Handy Guide to Wine and Steak Pairing | Wine Folly The Handy Guide to Wine and Steak Pairing With so many different cuts and ways to prepare a steak, you can discover a few go-to dry red wines or you can dive deep into pairing nuance. The rule of thumb when pairing with steak is to choose dry red wines – leaner cuts of meat pair with lighter wines, while richer, fattier cuts pair up with high tannin wines that can cut through the fat. Why they work: The Strip is flavorful and a thicker grain cut of beef, making it useful for several different methods of cooking, but it needs a wine that can complement the flavor AND cut through the fat.", "score": 0

In [28]:
pprint(tool_message.content)


('{"query": "wine pairing with steak", "follow_up_questions": null, "answer": '
 'null, "images": [], "results": [{"title": "The Handy Guide to Wine and Steak '
 'Pairing", "url": '
 '"https://winefolly.com/wine-pairing/the-handy-guide-to-wine-and-steak-pairing/", '
 '"content": "The Handy Guide to Wine and Steak Pairing | Wine Folly The Handy '
 'Guide to Wine and Steak Pairing With so many different cuts and ways to '
 'prepare a steak, you can discover a few go-to dry red wines or you can dive '
 'deep into pairing nuance. The rule of thumb when pairing with steak is to '
 'choose dry red wines – leaner cuts of meat pair with lighter wines, while '
 'richer, fattier cuts pair up with high tannin wines that can cut through the '
 'fat. Why they work: The Strip is flavorful and a thicker grain cut of beef, '
 'making it useful for several different methods of cooking, but it needs a '
 'wine that can complement the flavor AND cut through the fat.", "score": '
 '0.92763275, "raw_conten

In [29]:
pprint(tool_message.tool_call_id)

'call_ZD6L5LWCWGWb5d7JjhMz3rYC'


In [30]:
pprint(tool_message.name)

'tavily_search'


In [31]:
ai_msg.tool_calls

[{'name': 'tavily_search',
  'args': {'query': 'wine pairing with steak'},
  'id': 'call_ZD6L5LWCWGWb5d7JjhMz3rYC',
  'type': 'tool_call'}]

In [32]:
# batch 실행 - 도구 호출이 여러 개인 경우

# tool_messages = web_search.batch([tool_call])

tool_messages = web_search.batch(ai_msg.tool_calls)

print(tool_messages)
print("-" * 100)
pprint(tool_messages[0].content)

[ToolMessage(content='{"query": "wine pairing with steak", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "The Handy Guide to Wine and Steak Pairing", "url": "https://winefolly.com/wine-pairing/the-handy-guide-to-wine-and-steak-pairing/", "content": "The Handy Guide to Wine and Steak Pairing | Wine Folly The Handy Guide to Wine and Steak Pairing With so many different cuts and ways to prepare a steak, you can discover a few go-to dry red wines or you can dive deep into pairing nuance. The rule of thumb when pairing with steak is to choose dry red wines – leaner cuts of meat pair with lighter wines, while richer, fattier cuts pair up with high tannin wines that can cut through the fat. Why they work: The Strip is flavorful and a thicker grain cut of beef, making it useful for several different methods of cooking, but it needs a wine that can complement the flavor AND cut through the fat.", "score": 0.92763275, "raw_content": null}, {"title": "A Starter 

`(4) ToolMessage를 LLM에 전달하여 답변을 생성하기`

* 파라미터 이름이 config여야 하는 이유: @chain 데코레이터가 config라는 이름만 특별히 인식하도록 설계되었습니다.
* config 생략 가능한 이유: @chain이 자동으로 기본 RunnableConfig()를 생성해서 전달하기 때문입니다.

In [33]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
tavily_tool = TavilySearchResults(max_results=2)

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[tavily_tool])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)

    tool_msgs = tavily_tool.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
query = "스테이크와 어울리는 와인을 추천해주세요."
response = web_search_chain.invoke(query)

# 응답 출력 
pprint(response.content)

ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_zA8R3kFNFp4a42Cq08crmUcd', 'function': {'arguments': '{"query":"스테이크 와인 추천"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 109, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiwmareKmv9GzGOnbnuikoCnywqZG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--2b03d356-1970-4620-af54-92c2072aef7d-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '스테이크 와인 추천'}, 'id': 'call_zA8R3kFNFp4a42Cq08crmUcd', 'type': 'tool_call'}] usage_metadata={'input_tokens': 109, 'output_tokens': 23, 'total_toke

### 2-2. 사용자 정의 도구
- @tool decorator를 통해 사용자 정의 도구를 정의할 수 있음

`(1) 도구(tool) 정의하기`

In [34]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool

# Tool 정의 
@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

### 2-3. Runnable 객체를 도구(tool) 변환
- 문자열이나 dict 입력을 받는 Runnable을 도구로 변환
- as_tool 메서드를 사용

- wikipedia 설치
```
    poetry add wikipedia
```
- [WikipediaLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.wikipedia.WikipediaLoader.html) 사용


In [35]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from typing import List

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하는 함수 
def search_wiki(input_data: dict) -> List[Document]:
    """Search Wikipedia documents based on user input (query) and return k documents"""
    query = input_data["query"]
    k = input_data.get("k", 2)  
    wiki_loader = WikipediaLoader(query=query, load_max_docs=k, lang="ko")
    wiki_docs = wiki_loader.load()
    return wiki_docs

# 도구 호출에 사용할 입력 스키마 정의 
class WikiSearchSchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")
    k: int = Field(2, description="The number of documents to return (default is 2)")

# RunnableLambda 함수를 사용하여 위키피디아 문서 로더를 Runnable로 변환 
runnable = RunnableLambda(search_wiki)
wiki_search = runnable.as_tool(
    name="wiki_search",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a specified number of documents. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSearchSchema
)

In [36]:
# 도구 속성
print("자료형: ")
print(type(wiki_search))
print("-"*100)

print("name: ")
print(wiki_search.name)
print("-"*100)

print("description: ")
pprint(wiki_search.description)
print("-"*100)

print("schema: ")
pprint(wiki_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_search
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a specified number of documents. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'k': {'default': 2,
                      'description': 'The number of documents to return '
                                     '(default is 2)',
                      'title': 'K',
                      'type': 'integer'},
                'q

In [37]:
# 위키 검색 실행
query = "파스타의 유래"
wiki_results = wiki_search.invoke({"query":query})

# 검색 결과 출력
for result in wiki_results:
    print(result)  
    print("-" * 100)  

page_content='투움바 파스타(영어: Toowoomba pasta)는 미국의 파스타 요리이며,
이름은 오스트레일리아의 도시인 터움바에서 유래되었다.


== 역사 ==
미국 아웃백 스테이크하우스의 메뉴로 만들어졌으나, 미국에서는 단종되었으며 한국과 브라질 등지에서는 메뉴로 남아 있다. 특히 한국에서 크게 인기를 얻었으며, 한국 매체나 유튜버에 의해 세계에 알려지면서 한국 요리로 소개되거나 외국의 한국 음식점에서 판매되기도 한다.


== 만들기 ==
페투치네 등 두꺼운 파스타에 고춧가루를 넣어 매콤한 크림 소스를 추가한 음식이다.


== 활용 ==
투움바 소스를 활용한 음식으로 투움바 떡볶이, 투움바 라면, 투움바 치킨 등이 있다.


== 같이 보기 ==
아라비아타


== 각주 ==' metadata={'title': '투움바 파스타', 'summary': '투움바 파스타(영어: Toowoomba pasta)는 미국의 파스타 요리이며,\n이름은 오스트레일리아의 도시인 터움바에서 유래되었다.', 'source': 'https://ko.wikipedia.org/wiki/%ED%88%AC%EC%9B%80%EB%B0%94_%ED%8C%8C%EC%8A%A4%ED%83%80'}
----------------------------------------------------------------------------------------------------
page_content='피치(이탈리아어: pici, 단수: picio 피초[*])는 이탈리아의 파스타이다. 손으로 말아서 만드는 굵은 파스타의 일종으로 스파게티면이 좀 더 굵어진 것으로 보면 된다. 토스카나주의 시에나 현에서 유래했으며 몬탈치노 지역에서는 pinci라고 부른다.
반죽은 보통 밀가루나 물로만 만든다. 달걀을 첨가하는 것은 선택적이며 가정에 따라 다르다.
밀가루 반죽을 두껍고 평평하게 밀어서 편 다음 기다란 조각으로 잘라낸다. 잘라낸 조각을 두 손바닥 사이에서 말기도 하고 테이블 위에 놓고 

In [38]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_search])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bt0izlNodsmBbnQfkF8QtQa2', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집"}', 'name': 'tavily_search_func'}, 'type': 'function'}, {'id': 'call_W4qlzqvRRJDCiW75oIxw49Cl', 'function': {'arguments': '{"query": "파스타의 유래"}', 'name': 'wiki_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 176, 'total_tokens': 234, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Biwmsaz96tviMRDPVw82h4931wsxm', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--0ab4b30f-192e-4d8e-8ed7-ad04e00cb95b-0', tool_calls=[{'name': 'tavily_search_func', 'args': {'query': '서울 강남 파스타 맛집'}, 'i

`(2) LCEL 체인`
- 위키피디아 문서를 검색하고 내용을 요약하는 체인

In [39]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

# LLM 및 요약 체인 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser() 
)

# 요약 테스트 
summarized_text = summary_chain.invoke({"query":"파스타의 유래"})
pprint(summarized_text)

('투움바 파스타는 미국의 파스타 요리로, 오스트레일리아의 투움바에서 유래되었다. 미국 아웃백 스테이크하우스에서 처음 만들어졌으나 현재는 '
 '한국과 브라질에서 인기를 끌고 있다. 이 요리는 두꺼운 파스타에 매콤한 크림 소스를 추가하여 만든다. 이와 관련된 다양한 요리도 '
 '존재한다.\n'
 '\n'
 '이탈리아 요리는 기원전 4세기부터 발전해온 남유럽의 요리로, 지역마다 특색이 다르다. 북부는 쌀과 유제품을, 남부는 올리브와 해산물을 '
 '주로 사용한다. 이탈리아 요리는 간결함과 재료의 질을 중시하며, CNN에 의해 세계 최고의 요리로 선정된 바 있다. 이 요리는 역사적으로 '
 '다양한 문화의 영향을 받아 발전해왔으며, 고대 그리스와 로마의 요리법이 포함되어 있다.')


In [40]:
# 도구 호출에 사용할 입력 스키마 정의 
class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

# as_tool 메소드를 사용하여 도구 객체로 변환
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

# 도구 속성
print("자료형: ")
print(type(wiki_summary))
print("-"*100)

print("name: ")
print(wiki_summary.name)
print("-"*100)

print("description: ")
pprint(wiki_summary.description)
print("-"*100)

print("schema: ")
pprint(wiki_summary.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_summary
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a summarized text. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'query': {'description': 'The query to search for in Wikipedia',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'WikiSummarySchema',
 'type': 'object'}
-----------------------------

In [41]:
# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_summary])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eg4VJH9Qh4nd9E2qxOrr6kiN', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집"}', 'name': 'tavily_search_func'}, 'type': 'function'}, {'id': 'call_kTcUgfX72PY5PcJ38vj2Cuwl', 'function': {'arguments': '{"query": "파스타"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 152, 'total_tokens': 207, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-Biwn5neBaFlF0Vt334aFzil1yOOdg', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7d527628-79a7-4f63-be36-d8060ea8a70d-0', tool_calls=[{'name': 'tavily_search_func', 'args': {'query': '서울 강남 파스타 맛집'}, 'id':

In [42]:
ai_msg.tool_calls[1]

{'name': 'wiki_summary',
 'args': {'query': '파스타'},
 'id': 'call_kTcUgfX72PY5PcJ38vj2Cuwl',
 'type': 'tool_call'}

In [43]:
# 도구 실행 
tool_message = wiki_summary.invoke(ai_msg.tool_calls[1])

print(tool_message)
print("-" * 100)
pprint(tool_message.content)

content='파스타는 이탈리아의 주요 밀 식품으로, 듀럼밀 세몰라와 물 또는 밀가루와 달걀로 만들어지며, 삶거나 구워서 먹는다. 이탈리아의 국민 음식으로 여겨지며, 역사적으로 그리스와 아랍 문화에서 기원한 것으로 보인다. 파스타는 건파스타와 생파스타로 나뉘며, 다양한 형태와 종류가 있다. 요리 방법으로는 삶거나 오븐에 구워내는 방식이 있으며, 소스와의 조화가 중요하다. 투움바 파스타는 미국의 요리로, 매콤한 크림 소스를 사용하며 한국에서 인기를 끌고 있다.' name='wiki_summary' tool_call_id='call_kTcUgfX72PY5PcJ38vj2Cuwl'
----------------------------------------------------------------------------------------------------
('파스타는 이탈리아의 주요 밀 식품으로, 듀럼밀 세몰라와 물 또는 밀가루와 달걀로 만들어지며, 삶거나 구워서 먹는다. 이탈리아의 국민 '
 '음식으로 여겨지며, 역사적으로 그리스와 아랍 문화에서 기원한 것으로 보인다. 파스타는 건파스타와 생파스타로 나뉘며, 다양한 형태와 종류가 '
 '있다. 요리 방법으로는 삶거나 오븐에 구워내는 방식이 있으며, 소스와의 조화가 중요하다. 투움바 파스타는 미국의 요리로, 매콤한 크림 '
 '소스를 사용하며 한국에서 인기를 끌고 있다.')


In [44]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[wiki_summary])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def wiki_summary_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)

    tool_msgs = wiki_summary.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = wiki_summary_chain.invoke("파스타의 유래에 대해서 알려주세요.")

# 응답 출력 
pprint(response.content)

ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_SOZrAaNdOdZfQ7h4QArGmxiO', 'function': {'arguments': '{"query":"파스타의 유래"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 120, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiwnGYscqcoJGZoeXpaBkj69EbzbK', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--4fcbdb0d-d28b-4634-b31d-76ba36c14afa-0' tool_calls=[{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_SOZrAaNdOdZfQ7h4QArGmxiO', 'type': 'tool_call'}] usage_metadata={'input_tokens': 120, 'output_tokens': 19, 'total_tokens': 139, 'input_token_details': {

### 2-4. 벡터저장소 검색기
- @tool decorator 사용

`(1) 문서 로드 및 인덱싱`

In [45]:
from langchain.document_loaders import TextLoader

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_menu.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

1


In [46]:
from langchain_core.documents import Document

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 시그니처 스테이크
내용:
1. 시그니처 스테이크
   • 가격: ₩35,000
   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스
   • 설명: 셰프의 특제 시그니처 메뉴로, ...

메뉴 번호: 2
메뉴 이름: 트러플 리조또
내용:
2. 트러플 리조또
   • 가격: ₩22,000
   • 주요 식재료: 이탈리아산 아르보리오 쌀, 블랙 트러플, 파르미지아노 레지아노 치즈
   • 설명: 크리미한 텍스처의 리조...


### Embedding을 하기 위해 Ollama Qwen2.5 설치
```
ollama run qwen2.5:1.5b

ollama ls
```

In [47]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="qwen2.5:1.5b") 

# FAISS 인덱스 생성
menu_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# FAISS 인덱스 저장 (선택사항)
menu_db.save_local("./db/menu_db")


# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 2},
)

# 쿼리 테스트
query = "시그니처 스테이크의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

검색 결과: 2개
메뉴 번호: 3
메뉴 이름: 연어 타르타르

메뉴 번호: 1
메뉴 이름: 시그니처 스테이크



- 와인 메뉴에 대해서도 같은 작업을 처리

In [48]:
# 와인 메뉴 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_wine.txt", encoding="utf-8")
documents = loader.load()

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")


# FAISS 인덱스 생성
wine_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# 병합된 DB 다시 저장
wine_db.save_local("./db/wine_db")

wine_retriever = menu_db.as_retriever(
    search_kwargs={'k': 2},
)

query = "스테이크와 어울리는 와인을 추천해주세요."
docs = wine_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 샤토 마고 2015
내용:
1. 샤토 마고 2015
   • 가격: ₩450,000
   • 주요 품종: 카베르네 소비뇽, 메를로, 카베르네 프랑, 쁘띠 베르도
   • 설명: 보르도 메독 지역의 프리미엄 ...

메뉴 번호: 2
메뉴 이름: 돔 페리뇽 2012
내용:
2. 돔 페리뇽 2012
   • 가격: ₩380,000
   • 주요 품종: 샤르도네, 피노 누아
   • 설명: 프랑스 샴페인의 대명사로 알려진 프레스티지 큐베입니다. 시트러스...
검색 결과: 2개
메뉴 번호: 10
메뉴 이름: 티라미수

메뉴 번호: 1
메뉴 이름: 시그니처 스테이크



`(2) 도구(tool) 정의하기`

In [49]:
from langchain_core.tools import tool
# menu db 벡터 저장소 로드
menu_db = FAISS.load_local(
    "./db/menu_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_menu_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(db_search_menu_func))
print("-"*100)

print("name: ")
print(db_search_menu_func.name)
print("-"*100)

print("description: ")
pprint(db_search_menu_func.description)
print("-"*100)

print("schema: ")
pprint(db_search_menu_func.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
db_search_menu_func
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized restaurant menu information from the '
 'encrypted database.\n'
 'Use this tool only for menu-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Securely retrieve and access authorized restaurant menu '
                'information from the encrypted database.\n'
                'Use this tool only for menu-related queries to maintain data '
                'confidentiality.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'db_search_menu_func',
 'type': 'object'}


In [50]:
from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

# wine db 벡터 저장소 로드
wine_db = FAISS.load_local(
    "./db/wine_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_wine_func(query: str) -> List[Document]:
   """
   Securely retrieve and access authorized restaurant wine information from the encrypted database.
   Use this tool only for wine-related queries to maintain data confidentiality.
   """
   docs = wine_db.similarity_search(query, k=2)
   if len(docs) > 0:
      return docs
   
   return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(db_search_wine_func))
print("-"*100)

print("name: ")
print(db_search_wine_func.name)
print("-"*100)

print("description: ")
pprint(db_search_wine_func.description)
print("-"*100)

print("schema: ")
pprint(db_search_wine_func.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
db_search_wine_func
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized restaurant wine information from the '
 'encrypted database.\n'
 'Use this tool only for wine-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Securely retrieve and access authorized restaurant wine '
                'information from the encrypted database.\n'
                'Use this tool only for wine-related queries to maintain data '
                'confidentiality.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'db_search_wine_func',
 'type': 'object'}


In [51]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[db_search_menu_func, db_search_wine_func])

# 도구 호출이 필요한 LLM 호출을 수행
query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 와인도 추천 해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_R72CMc9HOewPUV6w5faiftQZ', 'function': {'arguments': '{"query": "시그니처 스테이크"}', 'name': 'db_search_menu_func'}, 'type': 'function'}, {'id': 'call_UgevyDaeBYwtOiU3qa64NIdT', 'function': {'arguments': '{"query": "스테이크와 잘 어울리는 와인"}', 'name': 'db_search_wine_func'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 133, 'total_tokens': 197, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiwnckIG8rqHFtWzzdJwKSxI3b13k', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2a55a95d-c498-41bc-aa68-3ef27083c856-0', tool_calls=[{'name': 'db_search_menu_func', 'args': {'query': '시그니

`(3) 여러 개의 도구(tool) 호출하기`

In [52]:
tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
for tool in tools:
    print(tool.name)

tavily_search_func
wiki_summary
db_search_menu_func
db_search_wine_func


In [53]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 4개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_menu_func":
            tool_message = db_search_menu_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_wine_func":
            tool_message = db_search_wine_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = restaurant_menu_chain.invoke("시그니처 스테이크의 가격과 특징은 무엇인가요? 와인도 해주세요.")

# 응답 출력 
print(response.content)

db_search_menu_func: 
{'name': 'db_search_menu_func', 'args': {'query': '시그니처 스테이크'}, 'id': 'call_TaTughqQxb3NZrxCi11Dj55E', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
db_search_wine_func: 
{'name': 'db_search_wine_func', 'args': {'query': '시그니처 스테이크'}, 'id': 'call_8Yi4TEi4O5RYJQshjAxbppZa', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(id='bd849a9c-e5c1-45e3-a246-66eed6237e3a', metadata={'source': './data/restaurant_menu.txt', 'menu_number': 1, 'menu_name': '시그니처 스테이크'}, page_content='1. 시그니처 스테이크\\n   • 가격: ₩35,000\\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(id='680c7144-82a5-4def-bb11-b9bc6f7e03fc', metadata={'source': './da

In [54]:
# 체인 실행
response = restaurant_menu_chain.invoke("파스타 메뉴가 있나요? 파스타의 역사 또는 유래를 알려주세요.")

# 응답 출력 
print(response.content)

db_search_menu_func: 
{'name': 'db_search_menu_func', 'args': {'query': '파스타'}, 'id': 'call_QKxtd0cIRgSFAucg4V2cWWTm', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '파스타의 역사'}, 'id': 'call_ELTaOyAiA9131pW3L3XCzDyP', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(id='bd849a9c-e5c1-45e3-a246-66eed6237e3a', metadata={'source': './data/restaurant_menu.txt', 'menu_number': 1, 'menu_name': '시그니처 스테이크'}, page_content='1. 시그니처 스테이크\\n   • 가격: ₩35,000\\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(id='7dabc1be-2990-46d6-b09b-480137305be0', metadata={'source': './data/restaurant_menu.txt

## 3. Few-shot 프롬프팅 
- 각 도구의 용도를 구분하여 few-shot 예제로 제시

### 3-1. Few-shot 도구 호출

In [55]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("트러플 리조또의 가격과 특징, 그리고 어울리는 와인에 대해 알려주세요.", name="example_user"),
    AIMessage("메뉴 정보를 검색하고, 위키피디아에서 추가 정보를 찾은 후, 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_menu_func", "args": {"query": "트러플 리조또"}, "id": "1"}]),
    ToolMessage("트러플 리조또: 가격 ₩28,000, 이탈리아 카나롤리 쌀 사용, 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리", tool_call_id="1"),    
    AIMessage("트러플 리조또의 가격은 ₩28,000이며, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "트러플 리조또", "k": 1}, "id": "2"}]),
    ToolMessage("트러플 리조또는 이탈리아 요리의 대표적인 리조또 요리 중 하나로, 고급 식재료인 트러플을 사용하여 만든 크리미한 쌀 요리입니다. \
                주로 아르보리오나 카나롤리 등의 쌀을 사용하며, 트러플 오일이나 생 트러플을 넣어 조리합니다. 리조또 특유의 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 것이 특징입니다.", tool_call_id="2"),
    AIMessage("트러플 리조또의 특징에 대해 알아보았습니다. 이제 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_wine_func", "args": {"query": "트러플 리조또에 어울리는 와인"}, "id": "3"}]),
    ToolMessage("트러플 리조또와 잘 어울리는 와인으로는 주로 중간 바디의 화이트 와인이 추천됩니다. 1. 샤르도네: 버터와 오크향이 트러플의 풍미를 보완합니다. \
                2. 피노 그리지오: 산뜻한 산미가 리조또의 크리미함과 균형을 이룹니다. 3. 베르나차: 이탈리아 토스카나 지방의 화이트 와인으로, 미네랄리티가 트러플과 잘 어울립니다.", tool_call_id="3"),
    AIMessage("트러플 리조또(₩28,000)는 이탈리아의 대표적인 리조또 요리 중 하나로, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다.\
               주요 특징으로는 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 점입니다. 고급 식재료인 트러플을 사용해 풍부한 맛과 향을 내며, \
              주로 아르보리오나 카나롤리 등의 쌀을 사용합니다. 트러플 리조또와 잘 어울리는 와인으로는 중간 바디의 화이트 와인이 추천됩니다. \
              특히 버터와 오크향이 트러플의 풍미를 보완하는 샤르도네, 산뜻한 산미로 리조또의 크리미함과 균형을 이루는 피노 그리지오,\
               그리고 미네랄리티가 트러플과 잘 어울리는 이탈리아 토스카나 지방의 베르나차 등이 좋은 선택이 될 수 있습니다.", name="example_assistant"),
]

system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{query}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인도 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'db_search_menu_func', 'args': {'query': '스테이크'}, 'id': 'call_7WQaC2a2oWXYdaFoGOZeNmyU', 'type': 'tool_call'}
{'name': 'db_search_wine_func', 'args': {'query': '스테이크'}, 'id': 'call_xMMqygFkFlDR43efw8tVGMpR', 'type': 'tool_call'}


In [56]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_Rez1N13HrjwISPrwqAzCxDtH', 'type': 'tool_call'}
{'name': 'tavily_search_func', 'args': {'query': '서울 강남 파스타 맛집'}, 'id': 'call_41eSUM1Cr4T4D36TTIjIkh1x', 'type': 'tool_call'}


### 3-2. 답변 생성 체인 

In [57]:
from datetime import datetime
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        # [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_wine_func":
            tool_message = db_search_wine_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_menu_func":
            tool_message = db_search_menu_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해 주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

db_search_menu_func: 
{'name': 'db_search_menu_func', 'args': {'query': '스테이크'}, 'id': 'call_ljc1IMkQ6LzM3KsSIWDySuey', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
db_search_wine_func: 
{'name': 'db_search_wine_func', 'args': {'query': '스테이크'}, 'id': 'call_U15Y5xNu9PtzaSIG20Q7qUF7', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(id='7dabc1be-2990-46d6-b09b-480137305be0', metadata={'source': './data/restaurant_menu.txt', 'menu_number': 8, 'menu_name': '안심 스테이크 샐러드'}, page_content='8. 안심 스테이크 샐러드\\n   • 가격: ₩26,000\\n   • 주요 식재료: 소고기 안심, 루꼴라, 체리 토마토, 발사믹 글레이즈\\n   • 설명: 부드러운 안심 스테이크를 얇게 슬라이스하여 신선한 루꼴라 위에 올린 메인 요리 샐러드입니다. 체리 토마토와 파마산 치즈 플레이크로 풍미를 더하고, 발사믹 글레이즈로 마무리하여 고기의 풍미를 한층 끌어올렸습니다.'), Document(id='bd849a9c-e5c1-45e3-a246-66eed6237e3a', metadata={'source': './data/restaurant_menu.txt', 'men

In [58]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_aQyhhz9RbOM1lh2BZUlu26Rj', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
db_search_menu_func: 
{'name': 'db_search_menu_func', 'args': {'query': '서울 강남 파스타 맛집'}, 'id': 'call_lUJyEQ1OKrhII5nBmTGV9wBE', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content='투움바 파스타는 미국의 파스타 요리로, 오스트레일리아의 투움바에서 유래되었다. 미국 아웃백 스테이크하우스에서 처음 만들어졌으나 현재는 한국과 브라질에서 인기를 끌고 있다. 이 요리는 두꺼운 파스타에 매콤한 크림 소스를 추가하여 만든다. 이탈리아 요리는 기원전 4세기부터 발전해온 남유럽의 요리로, 지역마다 특색이 있으며 북부는 쌀과 유제품, 남부는 올리브와 해산물을 주로 사용한다. 이탈리아 요리는 간결함과 재료의 질을 중시하며, 2013년 CNN에서 세계 최고의 요리로 선정되었다.', name='wiki_summary', tool_call_id='call_aQyhhz9RbOM1lh2BZUlu26Rj'), ToolMessage(content="[Document(id='bd849a9c-e5c1-45e3-a246-66eed6237e3a', metadata={'source': './data/restaurant_menu.txt', 'menu_number': 1, '

## 4. LangChain Agent 사용
- 유의사항: 프롬프트에 "agent_scratchpad",  "input" 변수를 포함

In [59]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        You are an AI assistant providing restaurant menu information and general food-related knowledge. 
        Your main goal is to provide accurate information and effective recommendations to users.

        Key guidelines:
        1. For restaurant menu information, use the search_menu tool. This tool provides details on menu items, including prices, ingredients, and cooking methods.
        2. For general food information, history, and cultural background, utilize the wiki_summary tool.
        3. For wine recommendations or food and wine pairing information, use the search_wine tool.
        4. If additional web searches are needed or for the most up-to-date information, use the search_web tool.
        5. Provide clear and concise responses based on the search results.
        6. If a question is ambiguous or lacks necessary information, politely ask for clarification.
        7. Always maintain a helpful and professional tone.
        8. When providing menu information, describe in the order of price, main ingredients, and distinctive cooking methods.
        9. When making recommendations, briefly explain the reasons.
        10. Maintain a conversational, chatbot-like style in your final responses. Be friendly, engaging, and natural in your communication.


        Remember, understand the purpose of each tool accurately and use them in appropriate situations. 
        Combine the tools to provide the most comprehensive and accurate answers to user queries. 
        Always strive to provide the most current and accurate information.
        """)),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [60]:
# Tool calling Agent 생성
from langchain.agents import AgentExecutor, create_tool_calling_agent

tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
agent = create_tool_calling_agent(llm, tools, agent_prompt)

# AgentExecutor 생성 
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [61]:
# AgentExecutor 실행

query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."
agent_response = agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `db_search_menu_func` with `{'query': '시그니처 스테이크'}`


[Document(id='bd849a9c-e5c1-45e3-a246-66eed6237e3a', metadata={'source': './data/restaurant_menu.txt', 'menu_number': 1, 'menu_name': '시그니처 스테이크'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(id='680c7144-82a5-4def-bb11-b9bc6f7e03fc', metadata={'source': './data/restaurant_menu.txt', 'menu_number': 2, 'menu_name': '트러플 리조또'}, page_content='2. 트러플 리조또\n   • 가격: ₩22,000\n   • 주요 식재료: 이탈리아산 아르보리오 쌀, 블랙 트러플, 파르미지아노 레지아노 치즈\n   • 설명: 크리미한 텍스처의 리조또에 고급 블랙 트러플을 듬뿍 얹어 풍부한 향과 맛을 즐길 수 있는 메뉴입니다. 24개월 숙성된 파르미지아노 레지아노 치즈를 사용하여 깊은 맛을 더했으며, 주문 즉시 조리하여 최상의 상태로 제공됩니다.')]
Invoking: `db_search_wine_func` with `{'query': '스테이크'}`


[Document(id='cf5a52c1-77cc-43bc-b40d-0d26a10bf47f', metadata={'sou

In [62]:
pprint(agent_response)

{'input': '시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요.',
 'output': '**시그니처 스테이크 정보:**\n'
           '- **가격:** 35,000원\n'
           '- **주요 재료:** 쇠고기, 로즈메리, 그릴드 아스파라거스\n'
           '- **특징:** 이 스테이크는 부드럽고 풍미가 가득한 쇠고기를 사용하며, 로즈메리와 함께 구워져 독특한 향을 '
           '자아냅니다. 또한, 그릴드 아스파라거스가 곁들여져 있어 맛의 균형을 이룹니다.\n'
           '\n'
           '**스테이크와 어울리는 와인 추천:**\n'
           '1. **도미니크 라부르 1er Cru 2018**\n'
           '   - **가격:** 320,000원\n'
           '   - **맛의 특징:** 블랙체리, 산딸기, 스파이스 향이 조화롭게 어우러지는 복합적인 맛을 가지고 있습니다. 강한 '
           '풍미의 스테이크와 잘 어울리며, 와인의 깊이감과 풍부한 맛이 스테이크의 맛을 더욱 돋보이게 합니다.\n'
           '\n'
           '2. **그라지 2016**\n'
           '   - **가격:** 950,000원\n'
           '   - **맛의 특징:** 복합적인 아로마가 인상적인 와인으로, 블랙베리, 자두의 맛이 느껴지며, 부드럽고 긴 여운이 '
           '있는 맛입니다. 역시 강한 맛의 스테이크와 찰떡궁합입니다.\n'
           '\n'
           '이 두 와인은 시그니처 스테이크와의 조화가 뛰어나며, 특별한 식사를 더욱 특별하게 만들어 줄 것입니다!'}


## 5. Gradio 활용

In [65]:
import gradio as gr
from typing import List, Tuple

def answer_invoke(message: str, history: List[Tuple[str, str]]) -> str:
    try:
        # 채팅 기록을 AI에게 전달할 수 있는 형식으로 변환
        chat_history = []
        for human, ai in history:
            chat_history.append(HumanMessage(content=human))
            chat_history.append(AIMessage(content=ai))
        
        # agent_executor를 사용하여 응답 생성
        response = agent_executor.invoke({
            "input": message,
            "chat_history": chat_history[-2:]    # 최근 2개의 메시지 기록만을 활용 
        })
        
        # agent_executor의 응답에서 최종 답변 추출
        return response['output']
    except Exception as e:
        # 오류 발생 시 사용자에게 알리고 로그 기록
        print(f"Error occurred: {str(e)}")
        return "죄송합니다. 응답을 생성하는 동안 오류가 발생했습니다. 다시 시도해 주세요."

# 예제 질문 정의
example_questions = [
    "시그니처 스테이크의 가격과 특징을 알려주세요.",
    "트러플 리조또와 잘 어울리는 와인을 추천해주세요.",
    "해산물 파스타의 주요 재료는 무엇인가요? 서울 강남 지역에 레스토랑을 추천해주세요.",
    "채식주의자를 위한 메뉴 옵션이 있나요?"
]

# Gradio 인터페이스 생성
demo = gr.ChatInterface(
    fn=answer_invoke,
    title="레스토랑 메뉴 AI 어시스턴트",
    description="메뉴 정보, 추천, 음식 관련 질문에 답변해 드립니다.",
    examples=example_questions,
    theme=gr.themes.Soft()
)

# 데모 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...

Invoking: `wiki_summary` with `{'query': '해산물 파스타'}`


Pasta, an Italian staple made from durum wheat semolina mixed with water or eggs, is a key component of Italian cuisine. Its history dates back to ancient times, with references to similar dishes found in Greek and Arabic texts. Pasta can be categorized into dried (pasta secca) and fresh (pasta fresca), each with various shapes and cooking methods. Italian cuisine, evolving since the 4th century BC, showcases regional diversity, with northern Italy favoring rice and dairy, while the south emphasizes olive oil, tomatoes, and seafood. The simplicity and quality of ingredients are hallmarks of Italian cooking, which has gained international acclaim.
Invoking: `db_search_menu_func` with `{'query': '강남 해산물 파스타 레스토랑'}`


[Document(id='07f549c3-b26b-4258-b541-805fed47127f', metadata={'source': './data/restaurant_menu.txt', 'menu_number': 3, 'menu_name': '연어 타르타르'}, page_content='3. 연어 타르타르\n   • 가격

In [64]:
# 데모 종료
demo.close()

Closing server running on port: 7860
